In [11]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../../")

from experiment import Exp

from dataloaders import data_set,data_dict
import torch
import yaml
import os

# 参数设置

# 训练参数 
除了路径 其他不要变

In [12]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   
# TODO change the path as relative path
args.to_save_path     = r"E:\TECO_Works\Conference\ISWC2022\Run_logs"              
args.freq_save_path   = r"E:\TECO_Works\Conference\ISWC2022\Freq_data"
args.window_save_path = r"E:\TECO_Works\Conference\ISWC2022\Sliding_window"
args.root_path        = r"E:\datasets"


args.drop_transition  = False
args.datanorm_type    = "standardization" # None ,"standardization", "minmax"


args.batch_size       = 256                                                     
args.shuffle          = True
args.drop_last        = False
args.train_vali_quote = 0.90                                           


# training setting 
args.train_epochs            = 150

args.learning_rate           = 0.001  
args.learning_rate_patience  = 7
args.learning_rate_factor    = 0.1


args.early_stop_patience     = 15

args.use_gpu                 = True if torch.cuda.is_available() else False
args.gpu                     = 0
args.use_multi_gpu           = False

args.optimizer               = "Adam"
args.criterion               = "CrossEntropy"

## 数据参数



In [13]:
args.seed                             = 1


args.data_name                        =  "dg"

args.wavelet_filtering                = False
args.wavelet_filtering_regularization = False
args.wavelet_filtering_finetuning     = False
args.wavelet_filtering_finetuning_percent = 0.5
args.wavelet_filtering_learnable      = False
args.wavelet_filtering_layernorm      = False

args.regulatization_tradeoff          = 0
args.number_wavelet_filtering         = 6


args.difference       = False 
args.filtering        =  False
args.magnitude        =  False
args.weighted_sampler = False




args.pos_select       = None
args.sensor_select    = None


args.representation_type = "time"
args.exp_mode            = "LOCV"
if args.data_name      ==  "skodar":
    args.exp_mode            = "SOCV"
config_file = open('../../configs/data.yaml', mode='r')
data_config = yaml.load(config_file, Loader=yaml.FullLoader)
config = data_config[args.data_name]

args.root_path       = os.path.join(args.root_path,config["filename"])
args.sampling_freq   = config["sampling_freq"]
args.num_classes     =  config["num_classes"]
window_seconds       = config["window_seconds"]
args.windowsize      =   int(window_seconds * args.sampling_freq) 
args.input_length    =  args.windowsize
# input information
args.c_in            = config["num_channels"]


if args.difference:
    args.c_in = args.c_in*2

if args.wavelet_filtering :
    
    if args.windowsize%2==1:
        N_ds = int(torch.log2(torch.tensor(args.windowsize-1)).floor()) - 2
    else:
        N_ds = int(torch.log2(torch.tensor(args.windowsize)).floor()) - 2

    args.f_in            =  args.number_wavelet_filtering*N_ds+1
else:
    args.f_in            =  1



## 模型参数

In [14]:
args.filter_scaling_factor = 0.25
args.model_type              = "deepconvlstm"#"deepconvlstm"#"sahar" #"deepconvlstm"
exp = Exp(args)
exp.model

Use GPU: cuda:0
Build the DeepConvLSTM model!
Done!
Parameter : 26834
Set the seed as :  1


model_builder(
  (model): DeepConvLSTM(
    (conv_blocks): ModuleList(
      (0): ConvBlock(
        (conv1): Conv2d(1, 16, kernel_size=(5, 1), stride=(1, 1))
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(5, 1), stride=(2, 1))
      )
      (1): ConvBlock(
        (conv1): Conv2d(16, 16, kernel_size=(5, 1), stride=(1, 1))
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(5, 1), stride=(2, 1))
      )
    )
    (lstm_layers): ModuleList(
      (0): LSTM(144, 32, batch_first=True)
    )
    (dropout): Dropout(p=0.5, inplace=False)
    (fc): Linear(in_features=32, out_features=2, bias=True)
  )
)

In [15]:
# cross_channel_interaction_type = "attn",    # attn  transformer  identity
# cross_channel_aggregation_type = "filter",  # filter  naive  FC  "SFCC", "SFCF"
# temporal_info_interaction_type = "gru",     # gru  lstm  attn  transformer  identity  conv
# temporal_info_aggregation_type = "FC",      # naive  filter  FC  tnaive

# 实验

In [17]:
import numpy as np

np.mean([0.8555020, 0.8739420, 0.8522432, 0.7673374 , 0.8190781 , 0.9518211,  0.8913517 , 0.8047190])

0.8519993125

In [19]:

np.mean([ 0.8905129, 0.8840352, 0.9069813, 0.8726712, 0.8676236, 0.9613272, 0.9121716, 0.7744681])

0.8837238875

In [18]:
np.mean([ 0.6987884,0.6431839,0.7590658,0.4979536,0.5980370,0.4851604,0.6151191, 0.5547645,0.6080755,1.0000000])

0.64601482

In [19]:
np.mean([0.8089986, 0.8780885, 0.7865685,0.9163156,0.8521332, 0.8666962,0.6393050, 0.7001114,0.8852817,0.8208783])

0.8154377

In [15]:
# 如果我们设置为 


args.model_type              = "tinyhar"#"deepconvlstm"#"sahar" #"deepconvlstm"

args.cross_channel_interaction_type = "attn"
args.cross_channel_aggregation_type = "FC"
args.temporal_info_interaction_type = "lstm"
args.temporal_info_aggregation_type = "tnaive"

exp = Exp(args)

Use GPU: cuda:0
Build the TinyHAR model!
Done!
Parameter : 19124
Set the seed as :  1


In [8]:
exp.model

model_builder(
  (model): TinyHAR_Model(
    (layers_conv): ModuleList(
      (0): Sequential(
        (0): Conv2d(1, 26, kernel_size=(5, 1), stride=(2, 1))
        (1): ReLU(inplace=True)
        (2): BatchNorm2d(26, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Sequential(
        (0): Conv2d(26, 26, kernel_size=(5, 1), stride=(2, 1))
        (1): ReLU(inplace=True)
        (2): BatchNorm2d(26, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): Sequential(
        (0): Conv2d(26, 26, kernel_size=(5, 1), stride=(2, 1))
        (1): ReLU(inplace=True)
        (2): BatchNorm2d(26, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (3): Sequential(
        (0): Conv2d(26, 26, kernel_size=(5, 1), stride=(2, 1))
        (1): ReLU(inplace=True)
        (2): BatchNorm2d(26, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (channel_interaction): SelfAttention_intera

In [ ]:
exp.train()

 ----------------------- load all the data -------------------
-----------------------Sliding file are generated -----------------------
-----------------------Sliding file are generated -----------------------
================ LOCV Mode ====================
================ 10 CV ======================
================ the 0 th CV Experiment ================ 
Leave one Out Experiment : The 1 Part as the test
[-] Target sampling weights:  [0.0001465  0.000181   0.00097182 0.00123001 0.0004852  0.00060716]
Train data number :  17901
The number of classes is :  6
The input_length  is :  100
The channel_in is :  3
Validation data number :  1989
Test data number :  9947
================ Build the model ================ 
Build the TinyHAR model!
Epoch: 1 cost time: 6.82715916633606
VALI: Epoch: 1, Steps: 70 | Train Loss: 1.2729058  Vali Loss: 0.7474972 Vali Accuracy: 0.7088989  Vali weighted F1: 0.6287872  Vali macro F1 0.3314323 
Validation loss decreased (inf --> 0.747497).  Saving model 

VALI: Epoch: 27, Steps: 70 | Train Loss: 0.0782483  Vali Loss: 0.1462164 Vali Accuracy: 0.9527401  Vali weighted F1: 0.9531757  Vali macro F1 0.9406470 
EarlyStopping counter: 5 out of 15
Learning rate adjusting counter: 5 out of 7
Epoch: 28 cost time: 5.542252063751221
VALI: Epoch: 28, Steps: 70 | Train Loss: 0.0739500  Vali Loss: 0.1399270 Vali Accuracy: 0.9537456  Vali weighted F1: 0.9536217  Vali macro F1 0.9392925 
EarlyStopping counter: 6 out of 15
Learning rate adjusting counter: 6 out of 7
Epoch: 29 cost time: 5.368205547332764
VALI: Epoch: 29, Steps: 70 | Train Loss: 0.0705354  Vali Loss: 0.1600473 Vali Accuracy: 0.9467069  Vali weighted F1: 0.9464967  Vali macro F1 0.9320401 
EarlyStopping counter: 7 out of 15
Learning rate adjusting counter: 7 out of 7
Updating learning rate to 0.0001
Epoch: 30 cost time: 5.382217168807983
VALI: Epoch: 30, Steps: 70 | Train Loss: 0.0631562  Vali Loss: 0.1465612 Vali Accuracy: 0.9507290  Vali weighted F1: 0.9505358  Vali macro F1 0.9347125 
E

Epoch: 17 cost time: 5.203176736831665
VALI: Epoch: 17, Steps: 68 | Train Loss: 0.1122171  Vali Loss: 0.1636804 Vali Accuracy: 0.9471229  Vali weighted F1: 0.9478014  Vali macro F1 0.9320431 
EarlyStopping counter: 1 out of 15
Learning rate adjusting counter: 1 out of 7
Epoch: 18 cost time: 5.199176073074341
VALI: Epoch: 18, Steps: 68 | Train Loss: 0.1093650  Vali Loss: 0.1615020 Vali Accuracy: 0.9471229  Vali weighted F1: 0.9470786  Vali macro F1 0.9301633 
EarlyStopping counter: 2 out of 15
Learning rate adjusting counter: 2 out of 7
Epoch: 19 cost time: 5.195183515548706
VALI: Epoch: 19, Steps: 68 | Train Loss: 0.1049017  Vali Loss: 0.1693727 Vali Accuracy: 0.9517885  Vali weighted F1: 0.9515991  Vali macro F1 0.9361981 
EarlyStopping counter: 3 out of 15
Learning rate adjusting counter: 3 out of 7
Epoch: 20 cost time: 5.21917200088501
VALI: Epoch: 20, Steps: 68 | Train Loss: 0.1026317  Vali Loss: 0.1618201 Vali Accuracy: 0.9523069  Vali weighted F1: 0.9520046  Vali macro F1 0.93864

Epoch: 3 cost time: 5.32920503616333
VALI: Epoch: 3, Steps: 70 | Train Loss: 0.3673163  Vali Loss: 0.3185138 Vali Accuracy: 0.8732824  Vali weighted F1: 0.8601290  Vali macro F1 0.7832514 
new best score!!!!
Validation loss decreased (0.423039 --> 0.318514).  Saving model ...
new best score!!!!
Epoch: 4 cost time: 5.352209806442261
VALI: Epoch: 4, Steps: 70 | Train Loss: 0.2983401  Vali Loss: 0.2728438 Vali Accuracy: 0.8890585  Vali weighted F1: 0.8786298  Vali macro F1 0.8082935 
new best score!!!!
Validation loss decreased (0.318514 --> 0.272844).  Saving model ...
new best score!!!!
Epoch: 5 cost time: 5.344208717346191
VALI: Epoch: 5, Steps: 70 | Train Loss: 0.2567945  Vali Loss: 0.2603034 Vali Accuracy: 0.9033079  Vali weighted F1: 0.9024426  Vali macro F1 0.8526226 
new best score!!!!
Validation loss decreased (0.272844 --> 0.260303).  Saving model ...
new best score!!!!
Epoch: 6 cost time: 5.332096338272095
VALI: Epoch: 6, Steps: 70 | Train Loss: 0.2246638  Vali Loss: 0.2301792 

Epoch: 32 cost time: 5.3312060832977295
VALI: Epoch: 32, Steps: 70 | Train Loss: 0.0597818  Vali Loss: 0.1108422 Vali Accuracy: 0.9623410  Vali weighted F1: 0.9619700  Vali macro F1 0.9467390 
EarlyStopping counter: 4 out of 15
Learning rate adjusting counter: 4 out of 7
Epoch: 33 cost time: 5.335206508636475
VALI: Epoch: 33, Steps: 70 | Train Loss: 0.0598754  Vali Loss: 0.1207108 Vali Accuracy: 0.9613232  Vali weighted F1: 0.9618433  Vali macro F1 0.9459486 
EarlyStopping counter: 5 out of 15
Learning rate adjusting counter: 5 out of 7
Epoch: 34 cost time: 5.330205917358398
VALI: Epoch: 34, Steps: 70 | Train Loss: 0.0690537  Vali Loss: 0.1123272 Vali Accuracy: 0.9643766  Vali weighted F1: 0.9646252  Vali macro F1 0.9482671 
EarlyStopping counter: 6 out of 15
Learning rate adjusting counter: 6 out of 7
Epoch: 35 cost time: 5.333205938339233
VALI: Epoch: 35, Steps: 70 | Train Loss: 0.0590639  Vali Loss: 0.1043940 Vali Accuracy: 0.9664122  Vali weighted F1: 0.9663487  Vali macro F1 0.952

...

Epoch: 16 cost time: 5.362213134765625
VALI: Epoch: 16, Steps: 70 | Train Loss: 0.1068069  Vali Loss: 0.1773510 Vali Accuracy: 0.9416540  Vali weighted F1: 0.9422605  Vali macro F1 0.9239251 
new best score!!!!
Validation loss decreased (0.182938 --> 0.177351).  Saving model ...
new best score!!!!
Epoch: 17 cost time: 5.3562116622924805
VALI: Epoch: 17, Steps: 70 | Train Loss: 0.1060732  Vali Loss: 0.1753073 Vali Accuracy: 0.9452055  Vali weighted F1: 0.9455180  Vali macro F1 0.9291730 
new best score!!!!
Validation loss decreased (0.177351 --> 0.175307).  Saving model ...
new best score!!!!
Epoch: 18 cost time: 5.363212585449219
VALI: Epoch: 18, Steps: 70 | Train Loss: 0.1011079  Vali Loss: 0.1693792 Vali Accuracy: 0.9462202  Vali weighted F1: 0.9470636  Vali macro F1 0.9335422 
new best score!!!!
Validation loss decreased (0.175307 --> 0.169379).  Saving model ...
new best score!!!!
Epoch: 19 cost time: 5.364212989807129
VALI: Epoch: 19, Steps: 70 | Train Loss: 0.0923720  Vali Loss: 

Epoch: 45 cost time: 5.371206283569336
VALI: Epoch: 45, Steps: 70 | Train Loss: 0.0440075  Vali Loss: 0.1329834 Vali Accuracy: 0.9624556  Vali weighted F1: 0.9625626  Vali macro F1 0.9495362 
EarlyStopping counter: 7 out of 15
Learning rate adjusting counter: 7 out of 7
Updating learning rate to 0.0001
Epoch: 46 cost time: 5.36621356010437
VALI: Epoch: 46, Steps: 70 | Train Loss: 0.0333865  Vali Loss: 0.1240279 Vali Accuracy: 0.9619482  Vali weighted F1: 0.9621947  Vali macro F1 0.9485265 
EarlyStopping counter: 8 out of 15
Learning rate adjusting counter: 1 out of 7
Epoch: 47 cost time: 5.362212896347046
VALI: Epoch: 47, Steps: 70 | Train Loss: 0.0299027  Vali Loss: 0.1269198 Vali Accuracy: 0.9609335  Vali weighted F1: 0.9611268  Vali macro F1 0.9474744 
EarlyStopping counter: 9 out of 15
Learning rate adjusting counter: 2 out of 7
Epoch: 48 cost time: 5.413493394851685
VALI: Epoch: 48, Steps: 70 | Train Loss: 0.0287979  Vali Loss: 0.1298003 Vali Accuracy: 0.9619482  Vali weighted F1:

Epoch: 19 cost time: 5.582336187362671
VALI: Epoch: 19, Steps: 72 | Train Loss: 0.0961152  Vali Loss: 0.1388401 Vali Accuracy: 0.9555994  Vali weighted F1: 0.9557792  Vali macro F1 0.9382146 
new best score!!!!
Validation loss decreased (0.139907 --> 0.138840).  Saving model ...
new best score!!!!
Epoch: 20 cost time: 5.581893444061279
VALI: Epoch: 20, Steps: 72 | Train Loss: 0.0933058  Vali Loss: 0.1446958 Vali Accuracy: 0.9565861  Vali weighted F1: 0.9560724  Vali macro F1 0.9390242 
EarlyStopping counter: 1 out of 15
Learning rate adjusting counter: 1 out of 7
Epoch: 21 cost time: 5.583957195281982
VALI: Epoch: 21, Steps: 72 | Train Loss: 0.0899180  Vali Loss: 0.1422759 Vali Accuracy: 0.9546127  Vali weighted F1: 0.9543596  Vali macro F1 0.9371195 
EarlyStopping counter: 2 out of 15
Learning rate adjusting counter: 2 out of 7
Epoch: 22 cost time: 5.634339332580566
VALI: Epoch: 22, Steps: 72 | Train Loss: 0.0891735  Vali Loss: 0.1254414 Vali Accuracy: 0.9615195  Vali weighted F1: 0.9

Epoch: 49 cost time: 5.617314100265503
VALI: Epoch: 49, Steps: 72 | Train Loss: 0.0439312  Vali Loss: 0.1258248 Vali Accuracy: 0.9605328  Vali weighted F1: 0.9606944  Vali macro F1 0.9467258 
EarlyStopping counter: 9 out of 15
Learning rate adjusting counter: 2 out of 7
Epoch: 50 cost time: 5.646283864974976
VALI: Epoch: 50, Steps: 72 | Train Loss: 0.0444579  Vali Loss: 0.1274006 Vali Accuracy: 0.9620128  Vali weighted F1: 0.9622139  Vali macro F1 0.9490825 
EarlyStopping counter: 10 out of 15
Learning rate adjusting counter: 3 out of 7
Epoch: 51 cost time: 5.580773830413818
VALI: Epoch: 51, Steps: 72 | Train Loss: 0.0457678  Vali Loss: 0.1240060 Vali Accuracy: 0.9610261  Vali weighted F1: 0.9611925  Vali macro F1 0.9475830 
new best score!!!!
Validation loss decreased (0.124360 --> 0.124006).  Saving model ...
new best score!!!!
Epoch: 52 cost time: 5.592134952545166
VALI: Epoch: 52, Steps: 72 | Train Loss: 0.0453897  Vali Loss: 0.1288803 Vali Accuracy: 0.9625062  Vali weighted F1: 0.

Epoch: 10 cost time: 5.494242429733276
VALI: Epoch: 10, Steps: 71 | Train Loss: 0.1546761  Vali Loss: 0.2033706 Vali Accuracy: 0.9207528  Vali weighted F1: 0.9211714  Vali macro F1 0.8851521 
new best score!!!!
Validation loss decreased (0.204919 --> 0.203371).  Saving model ...
new best score!!!!
Epoch: 11 cost time: 5.508688449859619
VALI: Epoch: 11, Steps: 71 | Train Loss: 0.1449466  Vali Loss: 0.1826320 Vali Accuracy: 0.9385835  Vali weighted F1: 0.9383873  Vali macro F1 0.9096742 
new best score!!!!
Validation loss decreased (0.203371 --> 0.182632).  Saving model ...
new best score!!!!
Epoch: 12 cost time: 5.507453441619873
VALI: Epoch: 12, Steps: 71 | Train Loss: 0.1383675  Vali Loss: 0.1713804 Vali Accuracy: 0.9435364  Vali weighted F1: 0.9433850  Vali macro F1 0.9159199 
new best score!!!!
Validation loss decreased (0.182632 --> 0.171380).  Saving model ...
new best score!!!!
Epoch: 13 cost time: 5.517247676849365
VALI: Epoch: 13, Steps: 71 | Train Loss: 0.1269886  Vali Loss: 0

In [ ]:
dataset = data_dict[args.data_name](args)

In [ ]:
import numpy as np

In [ ]:
start_index = dataset.train_slidingwindows[0][1]
end_index   = dataset.train_slidingwindows[0][2]
sample_x_1    = dataset.data_x.iloc[start_index:end_index, 1:-1].values

start_index = dataset.train_slidingwindows[100][1]
end_index   = dataset.train_slidingwindows[100][2]
sample_x_2    = dataset.data_x.iloc[start_index:end_index, 1:-1].values

temp_1 = np.expand_dims(sample_x_1,0)
temp_2 = np.expand_dims(sample_x_2,0)
combined_x = np.concatenate([temp_1,temp_2],axis=0)
combined_x = np.expand_dims(combined_x,1)
combined_x = torch.tensor(combined_x).double().to(exp.device)
print(combined_x.shape)

In [ ]:
out = exp.model(combined_x)
out = out.cpu().detach().numpy()

In [ ]:
out.shape

In [ ]:
weight = exp.model.wave_conv.wavelet_conv.weight.cpu().detach().numpy()

In [ ]:
for n,p in exp.model.named_parameters():
    print(n)

In [ ]:
import matplotlib.pyplot as plt
index1 = 0
index2 = 0

plt.figure()
plt.plot(out[0,0,:,index2])
plt.plot(out[0,1,:,index2])
plt.plot(out[0,2,:,index2])
plt.plot(out[0,3,:,index2])

In [ ]:
# 如果我们设置为 
# args.wavelet_filtering_learnable      = True
# exp = Exp(args)